# Código do Trabalho Final do Curso BI Master - PUC-Rio

### Utilizando HugginFace - Pytorch

In [1]:
# biblioteca do hugginface
!pip install --upgrade pip
!pip install transformers
!pip install unidecode
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss]

     |████████████████████████████████| 2.1 MB 4.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.3/596.3 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 6.0 MB/s eta 0:00:00
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-36ka3mbw/farm-haystack_7ea3abce028647a58146ebfd7a506087
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-36ka3mb

In [2]:
## Importação das bibliotecas
import pandas as pd
import numpy as np
import string
import unidecode
import os, glob, re, sys, random, unicodedata, collections

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

plt.style.use('ggplot')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.feature_extraction.text import CountVectorizer

from tqdm import tqdm
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.tokenize import sent_tokenize , word_tokenize
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('vader_lexicon')
nltk.download('stopwords')
## criando lista de stopwords
STOP_WORDS = stopwords.words('portuguese') + list(string.punctuation)
STOP_WORDS.append('\n')

## Importação do BERT - Hugginface
from transformers import AutoModel, AutoTokenizer, BertTokenizer, BertForSequenceClassification, pipeline
import torch

## Bibliotecas para busca em PT

## Bibliotecas para buscas em EN (apenas para exibição mais didática)
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader, DensePassageRetriever, PreProcessor
from haystack.document_stores import FAISSDocumentStore
from haystack.pipelines import ExtractiveQAPipeline


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.


In [3]:
## Verificação da GPU utilizada
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [4]:
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")

INFO - haystack.telemetry -  Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry


In [5]:
## Montagem do Drive e Ponteiramento para o diretório
from google.colab import drive
drive.mount('/content/drive')

work_path = '/content/drive/MyDrive/Classroom/PosGraduacao/ProjetoFinal/'
os.chdir(work_path)

Mounted at /content/drive


In [6]:
random.seed(50)

files = []

for dirname, _, filenames in os.walk('data/machado-de-assis/raw/txt'):
  for filename in filenames:
    files.append(os.path.join(dirname, filename))

print('There are a total of {} files'.format(len(files)), '\n')

There are a total of 116 files 



In [7]:
# Seleção de 10 arquivos aleatórios
sample_books = random.sample(files,10)

docs_ = []
for fname in sample_books:
    with open(fname , "r") as file:
        text = file.read()
        #todo verificar a formatação para excluir as quebras de linhas/parágrafos '\n'
    docs_.append(text)

# count term frequency using CountVectorizer from scikit-learn
## limiting number of words just for illustrating the concept
vec = CountVectorizer(max_features=10, stop_words=STOP_WORDS) 
X = vec.fit_transform(docs_)
df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names_out()) #get_feature_names())
books_names = [book.split('/')[-1] for book in sample_books]
df['book'] = books_names
df = df.set_index('book')

print(df)

                           ainda  casa  dia  disse  gilliatt  homem  ser  \
book                                                                       
gazeta.txt                    14    23   36     16         0     17   28   
cartaImprensa.txt              1     0    0      0         0      0    1   
inspiracoesClaustro.txt        1     0    0      1         0      4    8   
historiasMeiaNoite.txt        84    68   59    167         0     58   67   
vicondeCastilho.txt            1     0    0      0         0      0    0   
trabalhadoresMar.txt         121   210  173    146       776    312  179   
instintoNacionalidade.txt     17     2    2      3         0      3    4   
aoAcaso.txt                  140    67  123     34         0     45  167   
ressurreicao.txt              82    80   52    157         0     49   63   
henriqueChaves.txt             2     0    4      0         0      1    1   

                           tempo  todos  tudo  
book                                   

In [8]:
# Let's first get some files that we want to use
doc_dir = "data/machado-de-assis/raw/txt"
# Convert files to dicts
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(docs)

INFO - haystack.utils.preprocessing -  Converting data/machado-de-assis/raw/txt/cr+¡tica/ea.txt
INFO - haystack.utils.preprocessing -  Converting data/machado-de-assis/raw/txt/cr+¡tica/peregrinacao.txt
INFO - haystack.utils.preprocessing -  Converting data/machado-de-assis/raw/txt/cr+¡tica/contituinteSombraLuz.txt
INFO - haystack.utils.preprocessing -  Converting data/machado-de-assis/raw/txt/cr+¡tica/miragens.txt
INFO - haystack.utils.preprocessing -  Converting data/machado-de-assis/raw/txt/cr+¡tica/cenasVida.txt
INFO - haystack.utils.preprocessing -  Converting data/machado-de-assis/raw/txt/cr+¡tica/novaGeracao.txt
INFO - haystack.utils.preprocessing -  Converting data/machado-de-assis/raw/txt/cr+¡tica/pareceresConservatorioDramatico.txt
INFO - haystack.utils.preprocessing -  Converting data/machado-de-assis/raw/txt/cr+¡tica/cantosFantasias.txt
INFO - haystack.utils.preprocessing -  Converting data/machado-de-assis/raw/txt/cr+¡tica/floresFrutos.txt
INFO - haystack.utils.preprocessin

Writing Documents:   0%|          | 0/116 [00:00<?, ?it/s]

In [ ]:
for arquivo in docs:
  arquivo.content = arquivo.content.replace('\n', ' ')

In [17]:
import pathlib
from pathlib import Path

# apontando para o modelo treinado
dest = pathlib.Path.cwd()/'/content/drive/MyDrive/Classroom/PosGraduacao/ProjetoFinal/Modelo/HFmodels/neuralmind-bert-base-portuguese-cased-finetuned-squad-v1.1-pt'
#pierreguillou/bert-base-cased-squad-v1.1-portuguese
modelo_qea = str(dest)
nlp = pipeline("question-answering", model='pierreguillou/bert-base-cased-squad-v1.1-portuguese')

In [20]:
question = input('Digite sua pergunta:')

resultados = pd.DataFrame(columns=['Documento','Resposta','Score','Início','Fim'])
count = 0

for documento in docs:
  result = nlp(question=question, context=documento.content)
  resultados.loc[i] = [documento.meta['name'], result['answer'], result['score'], result['start'], result['end']]
  i += 1

#result = nlp(question=question, context=a)
print(question)
resultados.head()

#print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:703: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/pipelines/question_answering.py:300: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for span_id in range(num_spans)


Answer: 'Joaquim Serra', score: 0.993, start: 1446, end: 1459


## Daqui pra baixo é uma versão no qual estamos testando possíveis ajustes afim de funcionar em PT-BR

In [ ]:
# preprocessor = PreProcessor(
#     language="pt",
#     clean_empty_lines=True,
#     clean_whitespace=True,
#     clean_header_footer=False,
#     split_by="word",
#     split_length=100,
#     split_respect_sentence_boundary=True,)

# # Preprocessamento do DB
# documentos = preprocessor.process(docs)

100%|██████████| 116/116 [00:04<00:00, 27.76docs/s]


In [ ]:
# Now, let's write the dicts containing documents to our DB.
#document_store.write_documents(docs)
# document_store.write_documents(documentos)

Writing Documents:   0%|          | 0/19288 [00:00<?, ?it/s]

In [ ]:
# retriever = DensePassageRetriever(
#     document_store=document_store,
#     query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
#     passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
#     max_seq_len_query=64,
#     max_seq_len_passage=256,
#     batch_size=16,
#     use_gpu=True,
#     embed_title=True,
#     use_fast_tokenizers=True,
# )
# Important:
# Now that after we have the DPR initialized, we need to call update_embeddings() to iterate over all
# previously indexed documents and update their embedding representation.
# While this can be a time consuming operation (depending on corpus size), it only needs to be done once.
# At query time, we only need to embed the query and compare it the existing doc embeddings which is very fast.
# document_store.update_embeddings(retriever)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/493 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded facebook/dpr-ctx_encoder-single-nq-base
INFO - haystack.document_stores.faiss -  Updating embeddings for 19178 docs...


Updating Embedding:   0%|          | 0/19178 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/9184 [00:00<?, ? Docs/s]

In [ ]:
#reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)
# reader = FARMReader(model_name_or_path="pierreguillou/bert-base-cased-squad-v1.1-portuguese", use_gpu=True)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find pierreguillou/bert-base-cased-squad-v1.1-portuguese locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/862 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded pierreguillou/bert-base-cased-squad-v1.1-portuguese


Downloading:   0%|          | 0.00/494 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 2 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0  
INFO - haystack.modeling.infer -  /w\   /w\ 
INFO - haystack.modeling.infer -  /'\   / \ 


In [ ]:
# pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
# You can configure how many candidates the reader and retriever shall return
# The higher top_k for retriever, the better (but also the slower) your answers.
# prediction = pipe.run(
#     query="Qual foi a causa da morte de Brás Cubas?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
# )

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:483: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 23.40 Batches/s]


In [ ]:
# print_answers(prediction, details="medium")


Query: Qual foi a causa da morte de Brás Cubas?
Answers:
[   {   'answer': 'pecados',
        'context': 'oje um casal de virtudes no mesmo espaço de chão que '
                   'sofreu um\n'
                   'casal de pecados. Amanhã pode lá dormir um eclesiástico, '
                   'depois um\n'
                   'assassino, depois um',
        'score': 0.8123836815357208},
    {   'answer': 'pneumonia',
        'context': 'e física e moral, e o corpo fazia-se-me planta, e pedra e\n'
                   'Morri de uma pneumonia; mas se lhe disser que foi menos a\n'
                   'pneumonia, do que uma idéia grandi',
        'score': 0.7883062660694122},
    {   'answer': 'corrupção',
        'context': 'via;\n'
                   'havia a candura dele, que era botão de flor, ainda '
                   'entrecerrado à corrupção da\n'
                   'Tal era o contraste desses dois caracteres, que a estrela '
                   'da viúva',
        'score': 0.38080117106

Os códigos abaixo estão em vias de melhorias para tratamento e processamento dos dados.
Já possível verificar que há um retorno dos dados porém sem um resultado plausível uma vez que o modelo utilizado está baseado no idioma inglês.

Serão realizados testes com o modelo de DPR fornecido pela [Haystack](https://haystack.deepset.ai/overview/intro) e com o modelo de Q&A em PT afim de verificar qual atenderá melhor o projeto

# Até o momento os código abaixo não foram utilizados

In [ ]:
# BERT Base
# tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
# model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')

#tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
#model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased')

# BERT Large
#tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased')
#model = AutoModel.from_pretrained('neuralmind/bert-large-portuguese-cased')

#tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased')
#model = BertForSequenceClassification.from_pretrained('neuralmind/bert-large-portuguese-cased')